# AiiDA control

In [ ]:
import ipywidgets as ipw
from IPython.display import clear_output, display

In [ ]:
import subprocess
from aiida import engine

class DaemonControlPage(ipw.VBox):
    def __init__(self, *args, **kwargs):
        self._daemon = engine.daemon.get_daemon_client()
        self._status = ipw.Output()

        # Start daemon.
        start_button = ipw.Button(description="Start daemon", button_style="info")
        start_button.on_click(self._start_daemon)

        # Stop daemon.
        stop_button = ipw.Button(description="Stop daemon", button_style="danger")
        stop_button.on_click(self._stop_daemon)

        # Restart daemon.
        restart_button = ipw.Button(description="Restart daemon", button_style="warning")
        restart_button.on_click(self._restart_daemon)

        self.info=ipw.HTML()
        self._update_status()
        super().__init__(children=[self.info, self._status, ipw.HBox([start_button, stop_button, restart_button])])

    def _restart_daemon(self, _=None):
        self._clear_status()
        self.info.value = "Restarting the daemon..."
        response = self._daemon.restart_daemon()
        self.info.value = ""
        self._update_status()
        return response

    def _start_daemon(self, _=None):
        self._clear_status()
        self.info.value = "Starting the daemon..."
        response = self._daemon.start_daemon()
        self.info.value = ""
        self._update_status()
        return response

    def _stop_daemon(self, _=None):
        self._clear_status()
        self.info.value = "Stopping the daemon..."
        response = self._daemon.stop_daemon()
        self.info.value = ""
        self._update_status()
        return response

    def _update_status(self, _=None):
        self._clear_status()
        with self._status:
            result = subprocess.run(["verdi", "daemon", "status"], capture_output=True, text=True)
            print(result.stdout, result.stderr)
    
    def _clear_status(self):
        with self._status:
            clear_output()



In [ ]:
import aiidalab_widgets_base as awb
from aiida import orm

class ProcessControlPage(ipw.HBox):
    def __init__(self, *args, **kwargs):
        child_calcs = orm.QueryBuilder().append(orm.WorkflowNode, tag="node").append(orm.ProcessNode, with_incoming="node", filters={'attributes': {'!has_key': 'sealed'}}, project='id').all(flat=True)
        query = orm.QueryBuilder().append(orm.ProcessNode, filters={'id': {'!in': child_calcs}, 'attributes':{'!has_key': 'sealed'}}, project='uuid')
        self.process_list = [awb.ProcessNodesTreeWidget(value=p) for p in query.all(flat=True)]
        for process in self.process_list:
            process.observe(self._on_select, names="selected_nodes")
        self._selected = ipw.HTML()

        # Pause process.
        pause_button = ipw.Button(description="Pause process", button_style="warning")
        #pause_button.on_click(self._pause_process)
        
        # Play process.
        play_button = ipw.Button(description="Play process", button_style="success")
        #play_button.on_click(self._play_process)

        # Kill process.
        kill_button = ipw.Button(description="Kill process", button_style="danger")
        #play_button.on_click(self._kill_process)

        super().__init__(children=[ipw.VBox(self.process_list + [self._selected]), ipw.VBox([pause_button, play_button, kill_button])])
    
    def _on_select(self, change=None):
        self._selected_uuid = change['new'][0].uuid
        self._selected.value = self._selected_uuid


In [ ]:
class GroupControlPage(ipw.VBox):
    def __init__(self):
        text = ipw.HTML("I am a Group Control Page")
        super().__init__(children=[text])


In [ ]:
import subprocess
class StatusControlPage(ipw.HTML):
    def __init__(self):
        print("AiiDA status")
        print(subprocess.run(["verdi", "status"], capture_output=True, text=True).stdout)
        super().__init__()


In [ ]:
from aiida import manage

class Profile(ipw.HBox):
    def __init__(self, profile):
        self.profile = profile
        self.name = ipw.HTML(f"""<font size="3"> * {self.profile.name}</font>""")
        self.make_default = ipw.Button(description="Make default", button_style="info")
        self.delete = ipw.Button(description="Delele", button_style="danger")
        super().__init__(children=[self.name, self.make_default, self.delete])


class ProfileControlPage(ipw.VBox):
    def __init__(self):
        text = ipw.HTML(value="<h3> List of profiles </h3>")
        children = [Profile(p) for p in manage.get_config().profiles]
        super().__init__(children=children)


In [ ]:
correspondance = {
    "Daemon": DaemonControlPage,
    "Group": GroupControlPage,
    "Process": ProcessControlPage,
    "Status": StatusControlPage,
    "Profile": ProfileControlPage,
}

toc = ipw.ToggleButtons(
    options=correspondance.keys(),
    value=None,
    orientation="vertical",
    layout=ipw.Layout(width='200px')
)



output = ipw.Output()

def update_output(value={"new": "Group"}):
    if value['new'] in correspondance:
        with output:
            clear_output()
            display(correspondance[value['new']]())


toc.observe(update_output, names="value")

toc.value = "Daemon"

In [ ]:
display(ipw.HBox([toc, output]))